<a href="https://colab.research.google.com/github/spencer18001/ml_zoomcamp_project_2024_midterm/blob/main/Notebook/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!wget -O player_statistics_cleaned_final.csv https://raw.githubusercontent.com/spencer18001/ml_zoomcamp_project_2024_midterm/refs/heads/main/Data/player_statistics_cleaned_final.csv

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mutual_info_score, root_mean_squared_error

In [ ]:
df_path = "player_statistics_cleaned_final.csv"
df = pd.read_csv(df_path)
df.head()

,TeamName,PlayerName,Position,Games,Win rate,KDA,Avg kills,Avg deaths,Avg assists,CSPerMin,...,Avg VWPM,GD@15,CSD@15,XPD@15,FB %,FB Victim,Penta Kills,Solo Kills,Country,FlashKeybind
0,Top Esports,369,Top,8,0.500,3.1,2.5,2.4,4.9,7.6,...,0.13,322,5,424,0.000,0.25,0,2,China,D
1,Dplus KIA,aiming,Adc,9,0.333,4.8,5.0,2.0,4.7,10.1,...,0.10,-112,1,97,0.333,0.00,0,2,South Korea,F
2,MAD Lions KOI,alvaro,Support,5,0.200,1.5,0.2,4.4,6.2,1.2,...,0.49,-3,2,-256,0.000,0.60,0,-,Spain,D
3,Team Liquid,apa,Mid,10,0.500,2.4,3.5,3.2,4.2,8.7,...,0.18,278,2,216,0.400,0.00,0,3,United States,D
4,PSG Talon,azhi,Top,5,0.200,2.3,2.2,3.0,4.8,7.5,...,0.18,-311,-10,-168,0.200,0.00,0,-,Taiwan,D


In [ ]:
df.shape

(81, 27)

In [ ]:
df.drop(columns=['TeamName', 'PlayerName'], inplace=True)
column_to_move = df.pop('Win rate')
df['Win rate'] = column_to_move

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Position       81 non-null     object 
 1   Games          81 non-null     int64  
 2   KDA            81 non-null     float64
 3   Avg kills      81 non-null     float64
 4   Avg deaths     81 non-null     float64
 5   Avg assists    81 non-null     float64
 6   CSPerMin       81 non-null     float64
 7   GoldPerMin     81 non-null     int64  
 8   KP%            81 non-null     float64
 9   DamagePercent  81 non-null     float64
 10  DPM            81 non-null     int64  
 11  VSPM           81 non-null     float64
 12  Avg WPM        81 non-null     float64
 13  Avg WCPM       81 non-null     float64
 14  Avg VWPM       81 non-null     float64
 15  GD@15          81 non-null     int64  
 16  CSD@15         81 non-null     int64  
 17  XPD@15         81 non-null     int64  
 18  FB %        

In [ ]:
df.isnull().sum()

,0
Position,0
Games,0
KDA,0
Avg kills,0
Avg deaths,0
Avg assists,0
CSPerMin,0
GoldPerMin,0
KP%,0
DamagePercent,0


In [ ]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=random_state)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=random_state)

df_full_train = pd.concat([df_train, df_val]).reset_index(drop=True)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

In [ ]:
dv = DictVectorizer(sparse=False)
feature_dict = df.iloc[:, :-1].to_dict(orient='records')
X = dv.fit_transform(feature_dict)
model = LinearRegression()
model.fit(X, df.iloc[:, -1].values)
y_pred = model.predict(X)
root_mean_squared_error(df.iloc[:, -1].values, y_pred)

0.047934922921401585

In [ ]:
df.iloc[0]

,0
Position,Top
Games,8
KDA,3.1
Avg kills,2.5
Avg deaths,2.4
Avg assists,4.9
CSPerMin,7.6
GoldPerMin,373
KP%,0.532
DamagePercent,0.219


In [ ]:
y_pred[0]

0.4404533177702851

In [ ]:
feature_importance = pd.DataFrame({
    'Feature': list(dv.get_feature_names_out()),
    'Coefficient': model.coef_
})

feature_importance['Abs_Coefficient'] = feature_importance['Coefficient'].abs()
feature_importance = feature_importance.sort_values(by='Abs_Coefficient', ascending=False)

print(feature_importance)

                  Feature   Coefficient  Abs_Coefficient
25          DamagePercent -1.425793e+00     1.425793e+00
34                    KP% -9.520084e-01     9.520084e-01
0                Avg VWPM  2.789712e-01     2.789712e-01
12        Country=Czechia  1.913055e-01     1.913055e-01
20         Country=Sweden -1.787733e-01     1.787733e-01
46           Solo Kills=5  1.768650e-01     1.768650e-01
8         Country=Belgium  1.553808e-01     1.553808e-01
15        Country=Germany -1.479177e-01     1.479177e-01
14         Country=France -1.418787e-01     1.418787e-01
1                Avg WCPM -1.219185e-01     1.219185e-01
26                   FB % -1.201160e-01     1.201160e-01
21         Country=Taiwan -1.173862e-01     1.173862e-01
47           Solo Kills=6 -1.155405e-01     1.155405e-01
7                CSPerMin -1.005484e-01     1.005484e-01
39       Position=Support -9.997234e-02     9.997234e-02
16         Country=Poland  9.233905e-02     9.233905e-02
3             Avg assists  8.72